In [1]:
from keras.utils import Sequence
import numpy as np
import pickle as pkl
import pandas as pd

Using TensorFlow backend.


# Load Data

In [2]:
loadfile = open('data/X_Train_Seq', 'rb')
X_Train_Seq = pkl.load(loadfile)
loadfile.close()

loadfile = open('data/X_Test_Seq', 'rb')
X_Test_Seq = pkl.load(loadfile)
loadfile.close()

loadfile = open('data/Y_Train_Seq', 'rb')
Y_Train_Seq = pkl.load(loadfile)
loadfile.close()

loadfile = open('data/TranID_Seq', 'rb')
TranID_Seq = pkl.load(loadfile)
loadfile.close()

# Model 1

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Masking, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf
import keras.backend as K

In [4]:
feature_num = len(X_Train_Seq[0][0])

In [5]:
def modify_mse(y_true, y_pred):
    fraud = tf.ones_like(y_pred) - tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    normal = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return 10*K.mean(fraud*fraud) + K.mean(normal*normal)

In [6]:
model = Sequential()
model.add(Masking(mask_value=-99, input_shape=(3, feature_num)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss=modify_mse, metrics=['acc'])
print(model.summary())




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 3, 263)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 263)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 50)             62800     
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 50)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3, 32)             1632      
____________________________________________________________

In [7]:
best_weights_filepath = "best1.hdf5"
callback = EarlyStopping(monitor="val_loss", patience=10, verbose=1, mode="auto")
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
model.fit(np.array(X_Train_Seq), np.array(Y_Train_Seq).reshape((-1,3,1)), validation_split=0.2, epochs=1000, batch_size=128, callbacks=[callback,saveBestModel])




Train on 246484 samples, validate on 61622 samples
Epoch 1/1000





246484/246484 [==============================] - 59s 240us/step - loss: 0.1426 - acc: 0.9396 - val_loss: 0.1371 - val_acc: 0.9561

Epoch 00001: val_loss improved from inf to 0.13712, saving model to best1.hdf5
Epoch 2/1000
246484/246484 [==============================] - 82s 334us/step - loss: 0.1287 - acc: 0.9411 - val_loss: 0.1321 - val_acc: 0.9055

Epoch 00002: val_loss improved from 0.13712 to 0.13214, saving model to best1.hdf5
Epoch 3/1000
246484/246484 [==============================] - 87s 352us/step - loss: 0.1242 - acc: 0.9414 - val_loss: 0.1282 - val_acc: 0.9466

Epoch 00003: val_loss improved from 0.13214 to 0.12824, saving model to best1.hdf5
Epoch 4/1000
246484/246484 [==============================] - 87s 352us/step - loss: 0.1213 - acc: 0.9440 - val_loss: 0.1283 - val_acc: 0.9494

Epoch 00004: val_loss did not improve from 0.12824
Epoch 5/1000
246484/246484 [==============================] - 87s 352

246484/246484 [==============================] - 52s 213us/step - loss: 0.1001 - acc: 0.9505 - val_loss: 0.1235 - val_acc: 0.9487

Epoch 00032: val_loss improved from 0.12425 to 0.12346, saving model to best1.hdf5
Epoch 33/1000
246484/246484 [==============================] - 53s 215us/step - loss: 0.0998 - acc: 0.9507 - val_loss: 0.1249 - val_acc: 0.9361

Epoch 00033: val_loss did not improve from 0.12346
Epoch 34/1000
246484/246484 [==============================] - 53s 215us/step - loss: 0.0985 - acc: 0.9508 - val_loss: 0.1271 - val_acc: 0.9535

Epoch 00034: val_loss did not improve from 0.12346
Epoch 35/1000
246484/246484 [==============================] - 54s 218us/step - loss: 0.0988 - acc: 0.9507 - val_loss: 0.1242 - val_acc: 0.9336

Epoch 00035: val_loss did not improve from 0.12346
Epoch 36/1000
246484/246484 [==============================] - 53s 217us/step - loss: 0.0985 - acc: 0.9512 - val_loss: 0.1287 - val_acc: 0.9615

Epoch 00036: val_loss did not improve from 0.12346
Ep

# Test

In [8]:
def subFile(filename,threshold):
    predict = model.predict(np.array(X_Test_Seq)) > threshold
    index_TranID = 0
    answer = []
    for seqIndex,seq in enumerate(predict) :
        for tranIndex, tran in enumerate(seq):
            if TranID_Seq[index_TranID]!=-1:
                answer.append([int(TranID_Seq[index_TranID]),1 if tran else 0])
            index_TranID+=1
    answer = np.array(answer)
    a = {'TransactionID':answer[:,0],
          'isFraud':answer[:,1]}
    ans = pd.DataFrame(a)
    sample = pd.read_csv("data/IEEE/sample_submission.csv")
    sub = pd.merge(sample, ans, how="left", on="TransactionID")
    del sub['isFraud_x']
    sub = sub.rename(columns={'isFraud_y':'isFraud'})
    sub.to_csv('submission/sub1_'+filename+'.csv',index=False)

In [9]:
model.load_weights("best1.hdf5")

In [10]:
for t in ['.1','.2','.3','.4','.5','.6']:
    subFile(t,float(t))